# WhisperTS JSON vers SRT

In [71]:
import re,codecs,json
import pandas as pd

In [72]:
fTranscript="FrAr-00-22.json"
pause=0.200
with open(fTranscript) as inFile:
    whisper=json.load(inFile)

In [73]:
def second2timestamp(seconds):
    sHours=int(seconds/3600)
    sMinutes=int((seconds-sHours*3600)/60)
    sSeconds=int((seconds-sHours*3600-sMinutes*60))
    sMilliseconds=int((seconds%1)*1000)
    return "%02d:%02d:%02d,%03d"%(sHours,sMinutes,sSeconds,sMilliseconds)

In [74]:
def timestamp2second(timestamp):
    m=re.match("(\d\d):(\d\d):(\d\d),(\d\d\d)",timestamp)
    if m:
        seconds=(int(m.group(1))*60+int(m.group(2)))*60+int(m.group(3))+float(m.group(4))/1000
        return seconds
    else:
        print("problème de timestamp")
        return

In [75]:
def getWhisper(elements):
    lElements=[]
    if elements=="segments":
        for element in whisper[elements]:
            lElements.append((second2timestamp(element["start"]),
                              second2timestamp(element["end"]),
                              element["text"],
                              element["confidence"]))
    elif elements=="words":
        for segment in whisper["segments"]:
            for element in segment[elements]:
                lElements.append((second2timestamp(element["start"]),
                                  second2timestamp(element["end"]),
                                  element["text"],
                                  element["confidence"]))        
    return lElements

In [76]:
tMots=getWhisper("words")
dfMots=pd.DataFrame(tMots,columns="start end mot confidence".split(" "))
dfMots

,start,end,mot,confidence
0,"00:00:04,370","00:00:05,410","Spreadbury,",0.793
1,"00:00:05,469","00:00:06,049",il,0.972
2,"00:00:06,049","00:00:06,269",sera,0.999
3,"00:00:06,269","00:00:06,929",encadré,0.999
4,"00:00:06,929","00:00:07,030",par,0.999
...,...,...,...,...
3906,"00:22:19,039","00:22:19,059",le,0.988
3907,"00:22:19,059","00:22:19,579",capitaine.,0.992
3908,"00:22:20,099","00:22:20,240",C'est,0.986
3909,"00:22:20,240","00:22:20,279",le,0.989


In [77]:
dfSegments=pd.DataFrame(getWhisper("segments"),columns="start end mot confidence".split(" "))
dfSegments

,start,end,mot,confidence
0,"00:00:04,370","00:00:11,650","Spreadbury, il sera encadré par un Australien...",0.951
1,"00:00:13,380","00:00:19,019","M. Spreadbury, c'est le plus âgé des arbitres...",0.977
2,"00:00:20,609","00:00:29,269","et petit signe peut-être du destin, c'est lui...",0.967
3,"00:00:29,510","00:00:38,990",Après avoir demandé si tout le monde était pr...,0.972
4,"00:00:40,490","00:00:42,969","Allez, de suite dans le camp, Argentin.",0.913
...,...,...,...,...
524,"00:22:16,019","00:22:16,680",C'est le capitaine.,0.986
525,"00:22:16,900","00:22:18,079",C'est le capitaine.,0.986
526,"00:22:18,440","00:22:19,000",C'est le capitaine.,0.984
527,"00:22:19,019","00:22:19,579",C'est le capitaine.,0.985


In [78]:
second2timestamp(3665.547)

'01:01:05,547'

In [79]:
srtLines=[]
for i,(start,end,mot,confidence) in enumerate(tMots):
    n=i+1
    srtLines.append(str(n))
    srtLines.append(start+" --> "+end)
    srtLines.append(mot)
    srtLines.append("")


In [80]:
with codecs.open("0-mots.srt","w",encoding="utf8") as outFile:
    for srtLine in srtLines:
        outFile.write(srtLine+"\n")

In [81]:
srtPhrases=[]
srtMots=[]
bPhrase=True
pStart=0
pEnd=0
for i,(start,end,mot,confidence) in enumerate(tMots):
    
    if bPhrase:
        print ("start",start)
        pStart=start
        srtMots=[]
        bPhrase=False
    m=re.search(r"[.?!]$",mot)
    if m:
        print("end",end)
        pEnd=end
        srtMots.append(mot)
        pPhrase=" ".join(srtMots)
        srtPhrases.append((pStart,pEnd,pPhrase))
        bPhrase=True
    elif not bPhrase:
        srtMots.append(mot)

start 00:00:04,370
end 00:00:07,870
start 00:00:07,969
end 00:00:11,009
start 00:00:11,269
end 00:00:11,650
start 00:00:13,380
end 00:00:13,460
start 00:00:13,599
end 00:00:29,269
start 00:00:29,510
end 00:00:38,990
start 00:00:40,490
end 00:00:42,969
start 00:00:43,189
end 00:00:52,990
start 00:00:53,969
end 00:01:00,509
start 00:01:00,829
end 00:01:05,629
start 00:01:05,989
end 00:01:17,049
start 00:01:19,430
end 00:01:23,069
start 00:01:23,810
end 00:01:31,090
start 00:01:31,849
end 00:01:32,650
start 00:01:33,209
end 00:01:34,109
start 00:01:34,170
end 00:01:35,290
start 00:01:36,879
end 00:01:40,359
start 00:01:40,900
end 00:01:43,180
start 00:01:43,540
end 00:01:46,599
start 00:01:46,959
end 00:01:53,180
start 00:01:53,439
end 00:01:55,060
start 00:01:55,439
end 00:01:57,560
start 00:01:57,640
end 00:01:59,219
start 00:01:59,719
end 00:02:02,459
start 00:02:02,540
end 00:02:11,360
start 00:02:12,379
end 00:02:15,319
start 00:02:16,520
end 00:02:17,240
start 00:02:20,990
end 00:02

In [82]:
srtPhrases


[('00:00:04,370',
  '00:00:07,870',
  'Spreadbury, il sera encadré par un Australien, M.'),
 ('00:00:07,969',
  '00:00:11,009',
  'Dickinson, et par un Néo-Zélandais, [*] M.'),
 ('00:00:11,269', '00:00:11,650', 'Lawrence.'),
 ('00:00:13,380', '00:00:13,460', 'M.'),
 ('00:00:13,599',
  '00:00:29,269',
  "Spreadbury, c'est le plus âgé des arbitres de cette Coupe du Monde, le plus expérimenté, et petit [*] signe peut-être du destin, c'est lui qui dirigeait le France-Argentine, gagné par l'équipe de France au mois de novembre dernier, [*] ici même au Stade de France."),
 ('00:00:29,510',
  '00:00:38,990',
  "[*] Après avoir demandé si tout le monde était prêt, le coup d'envoi de la Coupe du Monde 2007 est donné par David Skrela."),
 ('00:00:40,490',
  '00:00:42,969',
  'Allez, de suite dans le camp, [*] Argentin.'),
 ('00:00:43,189',
  '00:00:52,990',
  'Avec un départ de Roncero, [*] Pichot a appelé le ballon [*] pour [*] Hernandez, [*] avec le pied droit, pour trouver une très jolie touc

In [83]:
srtPhrasesLines=[]
for i,(start,end,phrase) in enumerate(srtPhrases):
    n=i+1
    srtPhrasesLines.append(str(n))
    srtPhrasesLines.append(start+" --> "+end)
    srtPhrasesLines.append(phrase)
    srtPhrasesLines.append("")
    
with codecs.open("0-phrases.srt","w",encoding="utf8") as outFile:
    for srtPhrasesLine in srtPhrasesLines:
        outFile.write(srtPhrasesLine+"\n")

In [84]:
srtTours=[]
srtMots=[]
bTour=True
prevEnd=0
pStart=0
pEnd=0
for i,(start,end,mot,confidence) in enumerate(tMots):
    sStart=timestamp2second(start)
    sEnd=timestamp2second(end)
    if sStart-prevEnd>pause and prevEnd!=0:
        pEnd=prevEnd
        pTour=" ".join(srtMots)
        srtTours.append((pStart,pEnd,pTour))
        pStart=sStart
        prevEnd=sEnd
        srtMots=[mot]
        bPhrase=True
    else:
        srtMots.append(mot)
        prevEnd=sEnd
pTour=" ".join(srtMots)
srtTours.append((pStart,pEnd,pTour))

In [85]:
srtToursLines=[]
for i,(start,end,phrase) in enumerate(srtTours):
    n=i+1
    srtToursLines.append(str(n))
    srtToursLines.append(second2timestamp(start)+" --> "+second2timestamp(end))
    srtToursLines.append(phrase)
    srtToursLines.append("")
    
with codecs.open("0-tours.srt","w",encoding="utf8") as outFile:
    for srtToursLine in srtToursLines:
        outFile.write(srtToursLine+"\n")